<a href="https://colab.research.google.com/github/jikerbug/jibaek_project_generator/blob/master/%EC%95%85%EB%B3%B4%EC%A0%9C%EC%9E%91_%EB%B0%8F_%EC%BD%94%EB%93%9C%EB%8D%B0%EC%9D%B4%ED%84%B0%EB%B2%A0%EC%9D%B4%EC%8A%A4_%EA%B5%AC%EC%B6%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# class & 악보추출

import librosa
# pip install librosa

import vamp
"""
1. pip install vamp
2. https://code.soundsoftware.ac.uk/projects/vamp-plugin-pack 에서 운영체제에 맞는 exe파일을 받고 Chordino and NNLS chroma 플러그인만 설치
"""

import glob
import os.path
import youtube_dl

from PIL import Image, ImageDraw, ImageFont
#pip install pillow


class _Chord_Classification_Service:

    _instance = None
    
    # 유튜브 url로부터 시간별 코드리스트, 비트리스트, 유튜브 추출 파일 이름을 반환하는 함수
    def load_audio_data_from_url(self, url):
        ydl_opts = {
            'format': 'bestaudio',
        }
        with youtube_dl.YoutubeDL(ydl_opts) as ydl:
            ydl.download([url])

        # 확장자 변경
        files = glob.glob("*.webm")
        audio_file_name = ''
        for x in files:
            if not os.path.isdir(x):
                filename = os.path.splitext(x)
                try:
                    os.rename(x, filename[0] + '.wav')
                    audio_file_name = filename[0]
                except:
                    pass

        # load audio file
        signal, sr = librosa.load(filename[0] + '.wav')
        os.remove(audio_file_name + '.wav')

        predicted_beat_list = vamp.collect(signal, sr, "beatroot-vamp:beatroot")

        predicted_chord_list = vamp.collect(signal, sr, "nnls-chroma:chordino")

        chord_list = []
        for i in predicted_chord_list['list']:
            i['timestamp'] = float(i['timestamp'])
            chord_list.append(i)

        beat_list = []
        for i in predicted_beat_list['list']:
            beat_list.append(float(i['timestamp']))

        return chord_list, beat_list, audio_file_name.split('-')[0]
    
    # load_audio_data_from_url로부터 정보를 받아와 악보를 저장하는 함수
    def make_music_sheet(self, url, title):
        chord_list, beat_list, audio_file_name = self.load_audio_data_from_url(url)
        print(chord_list)
        print(beat_list)
        print(len(beat_list))

        # for문을 돌다가 마지막 element에 도달하기 전에 끊기지 않게 하기 위함
        while((len(beat_list) + 6) % 16 != 0):
            beat_list.append(beat_list[-1])
        print(len(beat_list))
        beat_list.append(beat_list[-1])

        text = ''
        if title == '':
            title = audio_file_name


        text += title
        text += '\n\n\n'


        total_chord_cnt = 0
        beat_flag = 0
        for chord in chord_list:
            if chord['timestamp'] >= 0 and chord['timestamp'] < beat_list[0]:
                if chord['label'] == 'N':
                    continue
                text += chord['label'] + '  '
                total_chord_cnt += 1

        init_flag = True
        for beat_id in range(len(beat_list)):
            if init_flag == False:
                if beat_flag != 16:
                    beat_flag += 1
                    continue
            else:
                if beat_flag != 10:
                    beat_flag += 1
                    continue
                init_flag = False

            for chord in chord_list:
                if chord['timestamp'] >= beat_list[beat_id - beat_flag] and chord['timestamp'] < beat_list[beat_id]:
                    text += chord['label'] + '  '
                    total_chord_cnt += 1
            text += '\n\n'
            beat_flag = 0

        for chord in chord_list:
            if chord['timestamp'] >= 0 and chord['timestamp'] < beat_list[0]:
                if chord['label'] == 'N':
                    continue
                text += chord['label'] + '  '
                total_chord_cnt += 1

        sheet_height = len(text.split('\n\n') )
        sheet_width = int(250 * total_chord_cnt/sheet_height)
        #./ batang.ttc
        print(sheet_height * 90 / sheet_width)
        if  sheet_height * 90 / sheet_width > 8:
            sheet_width += 250
        font = ImageFont.truetype('C:/Windows/Fonts/batang.ttc', 45)
        img = Image.new(mode='RGB', size=(sheet_width, sheet_height * 90), color='#FFF')
        draw = ImageDraw.Draw(img)
        draw.text((50, 50), text, font=font, fill='#000')
        img.save(title + '.png', format='PNG')

    def make_chord_database(self, filename):

        # 확장자 변경
        files = glob.glob(filename + '.webm')
        audio_file_name = ''
        for x in files:
            if not os.path.isdir(x):
                file_name = os.path.splitext(x)
                try:
                    os.rename(x, file_name[0] + '.wav')

                except:
                    pass

        print(filename)

        signal, sr = librosa.load(filename + '.wav')
        os.remove(filename + '.wav')

        predicted_beat_list = vamp.collect(signal, sr, "beatroot-vamp:beatroot")

        predicted_chord_list = vamp.collect(signal, sr, "nnls-chroma:chordino")

        chord_list = []
        for i in predicted_chord_list['list']:
            i['timestamp'] = float(i['timestamp'])
            chord_list.append(i)

        beat_list = []
        for i in predicted_beat_list['list']:
            beat_list.append(float(i['timestamp']))

        print(chord_list)
        print(beat_list)

        return chord_list, beat_list



def  Chord_Classification_Service():
    
    #singleton 구조인데 그냥 유튭강의 따라한 흔적. 속도 아주 조금더 빠르지 않을까 생각. 없애도 무방

    #ensure that we only have 1 instance of CCS
    if  _Chord_Classification_Service._instance is None:
        _Chord_Classification_Service._instance = _Chord_Classification_Service()
    return  _Chord_Classification_Service._instance

if __name__ == "__main__":

    ccs = Chord_Classification_Service()

    let_it_be_url = 'https://www.youtube.com/watch?v=QDYfEBY9NM4'
    The_visitor_url = 'https://www.youtube.com/watch?v=y5YmTj8KDWM'
    brown_url = 'https://www.youtube.com/watch?v=-5duYTCCtqI'

    title = ''
    ccs.make_music_sheet(brown_url, title)




In [ ]:
# main문 & 코드데이터베이스 구축

import youtube_dl
from chord_classification_service import Chord_Classification_Service


if __name__ == "__main__":

    css = Chord_Classification_Service()

    playlist_url = 'https://www.youtube.com/playlist?list=OLAK5uy_kfRVORKFdFjpkjalascOpqWbNawtinMmI'

    ydl = youtube_dl.YoutubeDL({
        'format': 'bestaudio',
        'dump_single_json': True,
        'extract_flat' : True
    })

    with ydl:
        info = ydl.extract_info(playlist_url, False)
    print(214)
    print(info)

    ydl = youtube_dl.YoutubeDL({
        'format': 'bestaudio'
    })

    with ydl:

        result = ydl.download([
            'https://www.youtube.com/playlist?list=OLAK5uy_kfRVORKFdFjpkjalascOpqWbNawtinMmI'])


    for music in info['entries']:
        filename = music['title'] + '-' + music['id']
        css.make_chord_database(filename)


